In [5]:
# connect mysql
import pymysql
conn = pymysql.connect(host = 'localhost', user='ys9925', password='1Q2w3e4r!@', db='crime_prediction', charset = 'utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)

In [6]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz  # DecisionTree 모듈, 시각화 모듈
from sklearn.model_selection import train_test_split  # train, test dataset 분류
from sklearn.model_selection import GridSearchCV # cross validation, hyper parameter
from collections import Counter # 라벨 분포 확인
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import pydot

In [7]:
# Season
for s in range(1, 5):
    if s == 1: season = 'Spring'
    elif s == 2: season = "Summer" 
    elif s == 3: season = "Fall" 
    else: season = "Winter"
        
    # Time
    for t in range(1, 5):
        if t == 1: time = "Morning"  
        elif t == 2: time = "Afternoon" 
        elif t == 3: time = "Evening" 
        else: time = "Midnight"
            
        # Weather
        # 1.맑음 2.흐림 3.눈/비 
        for w in range(1, 4):
            if w == 1: 
                where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud < 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud < 5);' 
            elif w == 2: 
                where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall = 0 and cloud >= 5) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall = 0 and cloud >= 5);'
            else: 
                where = 'WHERE (season = \''+ season + '\' and time = \'' + time + '\' and rainfall > 0) or (season = \'' + season + '\' and time = \'' + time + '\' and snowfall > 0);' 
                
            # select data
            dataset_sql = 'SELECT grid, CCTV_num, police_value, resident, floating FROM Dataset ' + where
            grid_sql = 'SELECT grid, CCTV_num, police_value, resident, floating FROM Grid WHERE time = \''+ time + '\''
            curs.execute(dataset_sql)
            dataset_result = curs.fetchall()
            curs.execute(grid_sql)
            grid_result = curs.fetchall()
            
            # Transform to Pandas Dataframe
            dataset_result = pd.DataFrame(dataset_result)
            grid_result = pd.DataFrame(grid_result)
            result = pd.concat([dataset_result, grid_result])
            result['crime_num'] = result.groupby(['grid']).grid.transform('count') -1
            result = result.drop_duplicates()
            result = result.sort_values('grid')
            print(result)
            
            np.set_printoptions(threshold=sys.maxsize)
            # Decision tree Classifier 생성

            # 학습해야하는 변수
            data = result[["CCTV_num", "police_value", "resident", "floating"]].to_numpy()
            # 맞춰야하는 변수: crime_num
            target = result.crime_num.to_numpy()

            # dataset 분할
            X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=10)

            # 하이퍼 파라미터 찾기
            tree = DecisionTreeClassifier()
            parameters = {'max_depth' : [x for x in range(3, 31, 1)], 'min_samples_leaf' : [x for x in range(1, 30, 1)], 'min_samples_split' : [x for x in range(2, 30, 1)]}

            # param_grid의  하이퍼 파라미터를 5개의 train, test set fold로 나누어 테스트 수행 설정
            ## refit=True가 기본 설정으로 True이면 가장 좋은 파라미터 설정으로 재학습 시킴
            grid_dtree = GridSearchCV(tree, param_grid=parameters, cv=5, refit=True)

            #학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습, 평가
            grid_dtree.fit(X_train, y_train)

            # GridSearchCV 결과를 추출해 데이터 프레임으로 반환
            scores_df = pd.DataFrame(grid_dtree.cv_results_)
            scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]
            
            print('GridSearch 최적 파라미터: ', grid_dtree.best_params_)
            print('GridSearch 최고 점수: ', grid_dtree.best_score_)

            # GridSearchCV의 refit으로 학습된 estimator 반환
            estimator = grid_dtree.best_estimator_

            # GridSearchCV의 best_estmator_ 는 이미 최적 학습이 됐으므로 별도 학습이 필요 없음
            pred = estimator.predict(X_test)

            # Confusion Matirx를 이용한 정확도 평가

            print('테스트 데이터세트 정확도: {0: .4f}'.format(accuracy_score(y_test, pred)))
            total = 0
            for i in range(pred.size):
                if pred[i] >= y_test[i] - 3 and pred[i] <= y_test[i] + 3:
                    total += 1
            accuracy = total/pred.size


            # predic test_set labels
            y_pred = estimator.predict(data)
            
            label = str(s - 1) + str(t - 1) + str(w - 1)
            print(label)
            print(y_pred)

            # 시각화를 위해 학습에 사용한 feature 라벨 추출
            feature_names = result.columns.tolist()
            feature_names = feature_names[1:5]
            
            file_name = label + "DecisionTree"
            
            # 트리 시각화
            dot_data = export_graphviz(estimator, out_file=file_name + ".dot", feature_names = feature_names, filled=True, rounded=True, special_characters=True)
            # 인코딩
            (graph,) = pydot.graph_from_dot_file(file_name + ".dot", encoding='utf8')
            # png로 저장
            graph.write_png(file_name + ".png") 

     grid  CCTV_num  police_value  resident  floating  crime_num
137     0         0             0        20       170          1
1       1         0             0        30       200          0
53      2         0             0        20       260          2
3       3         0             0        20       160          0
355     4         0             0        40       170          1
..    ...       ...           ...       ...       ...        ...
629  1102         8             0       310      1800          1
659  1103         0             0       330      1800          2
708  1104         0             0       450      1880          2
618  1105         0             0       240      1880          4
688  1106         0             0       280      1860          3

[1107 rows x 6 columns]


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearch 최적 파라미터:  {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 29}
GridSearch 최고 점수:  0.49943502824858765
테스트 데이터세트 정확도:  0.4775
000
[0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearch 최적 파라미터:  {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 29}
GridSearch 최고 점수:  0.5480225988700566
테스트 데이터세트 정확도:  0.5315
001
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0

...

GridSearch 최적 파라미터:  {'max_depth': 4, 'min_samples_leaf': 12, 'min_samples_split': 2}
GridSearch 최고 점수:  0.3954802259887006
테스트 데이터세트 정확도:  0.3604
330
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0
 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearch 최적 파라미터:  {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 16}
GridSearch 최고 점수:  0.5740112994350283
테스트 데이터세트 정확도:  0.5766
331
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearch 최적 파라미터:  {'max_depth': 3, 'min_samples_leaf': 24, 'min_samples_split': 2}
GridSearch 최고 점수:  0.816949152542373
테스트 데이터세트 정확도:  0.8784
332
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 